# MÓDULO 8: INTRODUCCIÓN A LANGGRAPH
## RESUMEN - SESIÓN 2: Diseño de Flujos y Multi-Agentes

Este notebook contiene una referencia rápida de los conceptos clave, patrones y comandos esenciales de la sesión.

---
## CONCEPTOS CLAVE

### 1. Ciclos de Refinamiento

**Concepto:** Permitir que un agente reintente una tarea hasta cumplir criterios de calidad.

**Analogía:** El lápiz con borrador vs la impresora.
- **LangChain:** Como una impresora, ejecuta una vez y termina
- **LangGraph:** Como un lápiz con borrador, puede corregir iterativamente

**Componentes necesarios:**

| Componente | Función | Ejemplo |
|------------|---------|------|
| **Nodo Generador** | Produce el output | `nodo_generar_tweet()` |
| **Nodo Validador** | Verifica criterios | `nodo_validar_longitud()` |
| **Router** | Decide reintentar/finalizar | `router_tweet()` |
| **Condición de salida** | Evita bucles infinitos | `intentos >= 3` |

In [ ]:
# Patrón de ciclo de refinamiento

def router(state) -> str:
    """Decide si continuar o finalizar el ciclo."""
    if state["criterio_cumplido"]:
        return "finalizar"
    elif state["intentos"] >= 3:  # Condición de escape
        return "finalizar"
    else:
        return "reintentar"

# Añadir edge condicional con ciclo
workflow.add_conditional_edges(
    "validador",
    router,
    {
        "reintentar": "generador",  # CICLO: vuelve al generador
        "finalizar": "finalizar"
    }
)

### 2. Persistencia con MemorySaver

**Concepto:** Guardar el estado del agente entre ejecuciones para mantener contexto.

**Analogía:** Guardar partida en un videojuego.
- Cada `thread_id` es una "ranura de guardado" independiente
- El estado persiste aunque cierres y reabras el juego

**Componentes necesarios:**

| Componente | Función | Código |
|------------|---------|--------|
| **MemorySaver** | Almacén de estados | `memory = MemorySaver()` |
| **thread_id** | Identificador único | `{"configurable": {"thread_id": "user_001"}}` |
| **checkpointer** | Activar persistencia | `compile(checkpointer=memory)` |

**Regla de oro:** Diferentes `thread_id` = estados completamente independientes

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Crear el almacén de estados
memory = MemorySaver()

# Compilar con persistencia
app = workflow.compile(checkpointer=memory)

# Ejecutar con thread_id único
config = {"configurable": {"thread_id": "sesion_001"}}
resultado1 = app.invoke(estado_inicial, config=config)

# Segunda ejecución con MISMO thread_id recuerda el estado anterior
resultado2 = app.invoke(nuevo_estado, config=config)

### 3. Multi-Agentes (Patrón Supervisor)

**Concepto:** Múltiples agentes especializados colaboran bajo coordinación de un supervisor.

**Analogía:** Equipo de marketing.
- **Copywriter:** Escribe contenido
- **Designer:** Crea diseño
- **SEO Specialist:** Optimiza keywords
- **Project Manager:** Coordina y aprueba (Supervisor)

**Arquitectura básica:**

```
┌──────────────┐
│  Supervisor  │ (Router que decide)
└──────┬───────┘
       │
   ┌───┴───┬───────┬────────┐
   │       │       │        │
┌──▼──┐ ┌──▼──┐ ┌──▼───┐ ┌──▼────┐
│Agent│ │Agent│ │Agent │ │ Final │
│  1  │ │  2  │ │  3   │ │       │
└─────┘ └─────┘ └──────┘ └───────┘
```

In [ ]:
# Patrón supervisor multi-agente

# Agente 1: Especialista en traducción
def nodo_traductor(state):
    return {"traduccion": resultado}

# Agente 2: Especialista en revisión
def nodo_revisor(state):
    return {"calidad_ok": True/False}

# Supervisor: Decide el flujo
def router_supervisor(state) -> str:
    """Coordina el trabajo de los agentes especializados."""
    if state["calidad_ok"]:
        return "publicar"
    elif state["intentos"] >= 3:
        return "publicar"
    else:
        return "retraducir"  # Ciclo de corrección

# Configurar edge condicional
workflow.add_conditional_edges(
    "revisor",
    router_supervisor,
    {
        "retraducir": "traductor",  # Vuelve al traductor
        "publicar": "publicar"
    }
)

### 4. Human-in-the-Loop (HITL)

**Concepto:** Pausar el agente para esperar aprobación humana antes de acciones críticas.

**Analogía:** Botón de pausa.
- El agente prepara todo
- Se detiene antes de ejecutar la acción final
- Espera el "OK" del humano para continuar

**Componentes necesarios:**

| Componente | Función | Código |
|------------|---------|--------|
| **interrupt_before** | Lista de nodos donde pausar | `interrupt_before=["publicar"]` |
| **checkpointer** | Obligatorio para pausas | `checkpointer=MemorySaver()` |
| **thread_id** | Identificar sesión pausada | `config={"configurable": {"thread_id": "..."}}` |

**Puntos de pausa comunes:**
- Antes de enviar emails
- Antes de realizar pagos
- Antes de publicar contenido
- Antes de modificar bases de datos

In [ ]:
# Patrón Human-in-the-Loop

# Compilar CON pausa
app = workflow.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["nodo_critico"]  # Pausa ANTES de este nodo
)

# Primera ejecución: se detendrá antes del nodo crítico
config = {"configurable": {"thread_id": "sesion_001"}}
resultado = app.invoke(estado_inicial, config=config)
# Estado: nodo_critico NO ejecutado aún

# Humano revisa y aprueba...

# Continuar ejecución (pasando None como input)
resultado_final = app.invoke(None, config=config)
# Ahora SÍ ejecuta el nodo_critico

---
## TABLA COMPARATIVA: LangChain vs LangGraph

| Característica | LangChain | LangGraph |
|----------------|-----------|--------|
| **Estructura** | Lineal (cadenas) | Grafo (ciclos permitidos) |
| **Flujo** | Secuencial fijo | Condicional dinámico |
| **Ciclos** | No nativos | Sí, con `add_conditional_edges` |
| **Persistencia** | No incorporada | Sí, con `MemorySaver` |
| **Human-in-the-Loop** | Manual | Nativo con `interrupt_before` |
| **Caso de uso** | Pipelines predecibles | Agentes complejos con decisiones |
| **Analogía** | Impresora | Lápiz con borrador |

---
## COMPONENTES FUNDAMENTALES

### Estado (State)

**Definición:** Diccionario tipado que viaja por el grafo.

**Reglas:**
- Cada nodo recibe el estado completo
- Cada nodo devuelve un `dict` con campos a actualizar
- Los campos no devueltos se mantienen sin cambios

In [ ]:
from typing import TypedDict

class MiEstado(TypedDict):
    campo1: str
    campo2: int
    campo3: bool

### Nodo

**Definición:** Función que transforma el estado.

**Principio:** 1 nodo = 1 responsabilidad

In [ ]:
def mi_nodo(state: MiEstado) -> dict:
    """Nodo que procesa el estado."""
    # Leer estado
    valor = state["campo1"]
    
    # Procesar
    nuevo_valor = procesar(valor)
    
    # Devolver SOLO lo que cambia
    return {"campo1": nuevo_valor}

### Edge (Conexión)

| Tipo | Cuándo usar | Sintaxis |
|------|-------------|----------|
| **Incondicional** | Siempre va de A a B | `add_edge("A", "B")` |
| **Condicional** | Decide destino según estado | `add_conditional_edges("A", router, mapping)` |

### Router

**Definición:** Función que decide el siguiente nodo.

**Devuelve:** Nombre del nodo destino (string)

In [ ]:
def mi_router(state: MiEstado) -> str:
    """Decide el siguiente nodo basándose en el estado."""
    if state["campo3"]:
        return "nodo_X"
    elif state["campo2"] > 10:
        return "nodo_Y"
    else:
        return "nodo_Z"

---
## PATRONES DE DISEÑO

### Patrón 1: Ciclo Simple

```
[Generador] → [Validador] → [Router]
     ↑                           |
     └──────(si falla)───────────┘
                                 |
                            (si OK)→ [FIN]
```

**Uso:** Refinamiento iterativo hasta cumplir criterio.

### Patrón 2: Ciclos Múltiples

```
[Generador] → [Validador 1] → [Router 1]
     ↑              |              |
     └─(falla V1)───┘         (OK V1)
                                   ↓
                         [Validador 2] → [Router 2]
                                ↑            |
                                └─(falla V2)─┘
                                             |
                                        (OK V2)→ [FIN]
```

**Uso:** Validaciones independientes que pueden fallar en diferentes momentos.

### Patrón 3: Supervisor Multi-Agente

```
         [Agente A]
              ↓
         [Agente B]
              ↓
        [Supervisor] ─→ (decide) ─→ [Siguiente paso]
              ↑
              └─────(corrección)─────┘
```

**Uso:** Coordinación de especialistas con ciclo de mejora.

### Patrón 4: Pausa Humana

```
[Preparar] → [Validar] → [PAUSA] → [Ejecutar] → [FIN]
                           ⏸
                     (espera humano)
```

**Uso:** Acciones críticas que requieren aprobación.

---
## COMANDOS ESENCIALES

### Construcción del Grafo

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# 1. Definir estado
class MiEstado(TypedDict):
    campo: str

# 2. Crear grafo
workflow = StateGraph(MiEstado)

# 3. Añadir nodos
workflow.add_node("nodo1", funcion1)
workflow.add_node("nodo2", funcion2)

# 4. Punto de entrada
workflow.set_entry_point("nodo1")

# 5. Conectar (incondicional)
workflow.add_edge("nodo1", "nodo2")

# 6. Conectar (condicional)
workflow.add_conditional_edges(
    "nodo2",
    mi_router,
    {
        "opcion_a": "nodo3",
        "opcion_b": "nodo4"
    }
)

# 7. Conectar a fin
workflow.add_edge("nodo3", END)

# 8. Compilar
app = workflow.compile()

### Persistencia

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Crear almacén
memory = MemorySaver()

# Compilar con persistencia
app = workflow.compile(checkpointer=memory)

# Ejecutar con thread_id
config = {"configurable": {"thread_id": "usuario_001"}}
resultado = app.invoke(estado_inicial, config=config)

### Human-in-the-Loop

In [ ]:
# Compilar con pausa
app = workflow.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["nodo_critico"]
)

# Ejecutar hasta pausa
config = {"configurable": {"thread_id": "sesion_001"}}
resultado = app.invoke(estado, config=config)

# Continuar tras aprobación
resultado_final = app.invoke(None, config=config)

### Visualización

In [ ]:
# Ver estructura del grafo
print(app.get_graph().draw_ascii())

---
## ERRORES COMUNES Y SOLUCIONES

### Error 1: Bucle Infinito

**Síntoma:** El agente se ejecuta indefinidamente.

**Causa:** Router sin condición de salida.

In [ ]:
# SOLUCIÓN: Siempre incluir condición de escape

def router(state) -> str:
    if state["criterio_ok"]:
        return "finalizar"
    elif state["intentos"] >= 3:  # ← Condición de escape obligatoria
        return "finalizar"
    else:
        return "reintentar"

### Error 2: Estado No Persiste

**Síntoma:** El agente no recuerda entre ejecuciones.

**Causa 1:** Falta `checkpointer`

In [ ]:
# MAL: Sin persistencia
app = workflow.compile()

# BIEN: Con persistencia
app = workflow.compile(checkpointer=MemorySaver())

**Causa 2:** thread_id diferente en cada ejecución

In [ ]:
# MAL: thread_id aleatorio cada vez
import random
config = {"configurable": {"thread_id": str(random.randint(1,1000))}}

# BIEN: thread_id consistente
config = {"configurable": {"thread_id": "usuario_001"}}

### Error 3: Pausa No Funciona

**Síntoma:** El grafo no se detiene en `interrupt_before`.

**Causa:** Falta `checkpointer`.

In [ ]:
# MAL: interrupt_before sin checkpointer
app = workflow.compile(interrupt_before=["nodo"])

# BIEN: Con checkpointer
app = workflow.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["nodo"]
)

### Error 4: Router Devuelve Nodo Inexistente

**Síntoma:** Error "Node 'X' not found"

**Causa:** String del router no coincide con nombre del nodo.

In [ ]:
# El nombre del nodo y el string del router DEBEN coincidir exactamente

# Definir nodo
workflow.add_node("mi_nodo", funcion)

# Router DEBE devolver exactamente "mi_nodo"
def router(state) -> str:
    return "mi_nodo"  # ← Mismo nombre, sensible a mayúsculas

---
## CHECKLIST DE DEPURACIÓN

Cuando tu grafo no funciona como esperas, verifica:

- [ ] ¿Todos los nodos devuelven `dict` (no `state` completo)?
- [ ] ¿El router devuelve `str` con nombres exactos de nodos?
- [ ] ¿Los ciclos tienen condición de salida?
- [ ] ¿Usas `checkpointer` si necesitas persistencia?
- [ ] ¿Usas el mismo `thread_id` entre ejecuciones relacionadas?
- [ ] ¿Usas `checkpointer` si usas `interrupt_before`?
- [ ] ¿Visualizaste el grafo con `draw_ascii()` para verificar estructura?

---
## RECURSOS ADICIONALES

**Documentación Oficial:**
- LangGraph: https://langchain-ai.github.io/langgraph/
- LangChain: https://python.langchain.com/

**Conceptos para Profundizar:**
- Streaming: Enviar actualizaciones en tiempo real
- Subgrafos: Grafos dentro de grafos
- Paralelización: Ejecutar múltiples nodos simultáneamente
- Herramientas externas: Integrar APIs y servicios